# Practicing own

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")

# loader

In [2]:
# data ingestion
from langchain_community.document_loaders import WebBaseLoader
loader= WebBaseLoader(web_path="https://python.langchain.com/docs/versions/v0_3/")
doc=loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


# splitting text

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)
splitter

In [4]:
split_doc=splitter.split_documents(doc)

In [5]:
len(split_doc)

111

# Embeddings and vector store

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embeddings=OpenAIEmbeddings(
    model='text-embedding-3-small'
)

vector_store=FAISS.from_documents(
    split_doc,embeddings
)
vector_store

# document chain and retriever chain

In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """Answer the following questions only based on context provided.
    <context>
    {context}
    </context>
    question: {input}
    """
)


In [16]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(
    model='gpt-3.5-turbo',
)

In [ ]:
# document chain : To give context to the prompt and get response from the llm
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [20]:
# retriver chain to get the response from the vector store
from langchain.chains import create_retrieval_chain
retriever=vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

retriever_chain=create_retrieval_chain(retriever,document_chain)

In [21]:
# get response from the llm
result=retriever_chain.invoke({
    "input":"what is the changes in the new version of langchain?"
})

In [22]:
result

{'input': 'what is the changes in the new version of langchain?',
 'context': [Document(id='50187e72-b7ba-4a7e-9528-e340d7ab4743', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='Any package that is now compatible with both langchain-core 0.2 and 0.3 had a patch version bump.\nYou can use the langchain-cli to update deprecated imports automatically.'),
  Document(id='6f9d03f1-e5aa-4ffb-90c0-6fd3f419cb4d', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='LangChain v0.3 | 🦜️🔗 LangChain'),
  Document(id='2f55f705-e45b-4868-bdf2-de8d03a42d25', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}

In [23]:
result['answer']

'The changes in the new version of LangChain include compatibility with both langchain-core 0.2 and 0.3 with a patch version bump.'